In [9]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import linalg

In [10]:
def cond_spectr(A: np.ndarray) -> float:
    return linalg.norm(A) * linalg.norm(linalg.inv(A))


def cond_vol(A: np.ndarray) -> float:
    dividend = 1.0
    for row in A:
        dividend = dividend * linalg.norm(row, 2)

    return dividend / abs(linalg.det(A))


def cond_angle(A: np.ndarray) -> float:
    A_inv = linalg.inv(A)
    max_val = 0
    for i in range(A.shape[0]):
        curr_val = linalg.norm(A[i]) * linalg.norm(A_inv[..., i])
        max_val = max(max_val, curr_val)

    return max_val


def print_conds(A: np.ndarray) -> None:
    print("Cond. spectr:\t", cond_spectr(A))
    print("Cond. volume:\t", cond_vol(A))
    print("Cond. angle:\t", cond_angle(A))

In [11]:
def M_i(i, U):
    matr = np.eye(order)
    for j in range(i, order):
        matr[j][i-1] = - U[j, i-1] / U[i-1, i-1]
    return matr

In [12]:
Z_ = np.random.uniform(-100, 100, (10, 10))
Z = linalg.hilbert(15)
Z_ = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]])
Z_ = np.array([[12.951443, 1.554567, -3.998582],\
              [1.554567, 9.835076, 0.930339],\
              [-3.998582, 0.930339, 7.80380]])
Z_ = np.array([[8.29381, 0.995516, -0.560617],
               [0.995516, 6.298198, 0.595772],
               [-0.560617, 0.595772, 4.997407]])
Z_ = np.array([[1, 9, 2, 1, 1],
               [10, 1, 2, 1, 1],
               [1, 0, 5, 1, 1],
               [2, 1, 1, 2, 9],
               [2, 1, 2, 13, 2]])

order = Z.shape[0]

U = Z.copy()
L = np.eye(order)
for i in range(1, order):
    curr_M = M_i(i, U)
    U = curr_M @ U
    L = L @ linalg.inv(curr_M)


print("||LU - Z||:", linalg.norm(L@U - Z))

print("Z:")
print_conds(Z)
print("L:")
print_conds(L)
print("U:")
print_conds(U)

||LU - Z||: 1.1978388074229732e-16
Z:
Cond. spectr:	 4.6845908193094054e+17
Cond. volume:	 5.122528618888359e+112
Cond. angle:	 3.4559620927942348e+16
L:
Cond. spectr:	 31942.450205248737
Cond. volume:	 2485402452299409.5
Cond. angle:	 2382.567956753683
U:
Cond. spectr:	 2.5474080891075656e+16
Cond. volume:	 9.320410906126957e+18
Cond. angle:	 1812.5916964382814


In [13]:
#x_rand = np.random.uniform(-1, 1, (order,))
x_ones = np.ones(order)
b = Z @ x_ones
y_ = linalg.solve_triangular(L, b, lower=True, unit_diagonal=True)
x = linalg.solve_triangular(U, y_, lower=False)
x_c = linalg.solve(Z, b)
#print(x_ones)
#print(x)
#print(x_c)
print(linalg.norm(x-x_ones) / linalg.norm(x_ones))
print(linalg.norm(x-x_c) / linalg.norm(x_ones))

5.594467094093577
10.106839574850753


/tmp/ipykernel_216787/4211035871.py:6: LinAlgWarning: Ill-conditioned matrix (rcond=8.18724e-19): result may not be accurate.
  x_c = linalg.solve(Z, b)


In [14]:
#p_, l_, u_ = linalg.lu(A)
lu, piv = linalg.lu_factor(Z)
x = linalg.lu_solve((lu, piv), b)

In [15]:
rows=[]
for deg in range(-12, 0):
    alpha = 10**deg
    Z_reg = Z + alpha*np.eye(order)
    x_curr = linalg.lu_solve(linalg.lu_factor(Z_reg), b+10**deg*x_ones)
    #x_curr = linalg.solve(A_reg, b+10**deg*x_rand)
    
    x_rand_other = np.random.uniform(-10, 10, (order,))
    b_other = Z @ x_rand_other
    x_other = linalg.lu_solve(linalg.lu_factor(Z_reg), b_other+alpha*x_rand_other)
    row=[alpha, linalg.norm(x_curr-x_ones), linalg.norm(x_other-x_rand_other)]
    rows.append(row)

df = pd.DataFrame(rows, columns=['alpha', 'err_ones', 'err_rand'])
df

,alpha,err_ones,err_rand
0,1.000000e-12,1.655783e-04,5.338953e-04
1,1.000000e-11,2.435666e-05,5.438817e-05
2,1.000000e-10,2.308208e-06,1.589250e-06
3,1.000000e-09,1.086279e-07,3.882104e-07
4,1.000000e-08,1.833401e-08,4.503460e-08
5,1.000000e-07,2.485011e-09,5.784214e-09
6,1.000000e-06,3.095773e-10,3.386120e-10
7,1.000000e-05,2.387703e-11,6.277172e-11
8,1.000000e-04,3.434609e-12,4.617774e-12
9,1.000000e-03,3.546696e-13,1.203952e-12
